<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 778kB 8.5MB/s 
     |████████████████████████████████| 1.1MB 25.2MB/s 
     |████████████████████████████████| 3.0MB 59.2MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtBert Model<b>

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

In [4]:
model = AutoModel.from_pretrained("Rostlab/prot_bert")

<b>3. Load the model into the GPU if avilabile<b>

In [5]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0)

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Extracting sequences' features and covert the output to numpy if needed<b>

In [8]:
embedding = fe(sequences_Example)

In [9]:
embedding = np.array(embedding)

In [10]:
print(embedding)

[[[ 0.04535879  0.11401558 -0.01174266 ... -0.08750971 -0.11434396
    0.02040843]
  [ 0.09233951  0.13914399 -0.05241546 ... -0.13947977 -0.0428006
    0.07431364]
  [ 0.11505969  0.01998261 -0.08627468 ... -0.00946233 -0.1872724
    0.13169006]
  ...
  [ 0.10792391  0.09771117 -0.05832333 ... -0.12765658 -0.06493297
    0.12894136]
  [ 0.0545795   0.03635893 -0.07821195 ... -0.03016043 -0.06015311
    0.08903892]
  [ 0.05150128  0.0571134  -0.06932486 ... -0.03937564 -0.06630812
    0.09767482]]

 [[ 0.06268476  0.08731148  0.00279815 ... -0.03485441 -0.037174
    0.01748304]
  [ 0.0880546   0.10885102  0.06212992 ...  0.0144938  -0.03327068
   -0.0362049 ]
  [-0.03332657  0.03442192  0.0809955  ...  0.0008738  -0.02733596
    0.05783575]
  ...
  [ 0.00774996  0.03800717 -0.00171148 ... -0.00877388 -0.02430492
    0.07620303]
  [ 0.02335456  0.03041686 -0.00720565 ...  0.00245539 -0.01678618
    0.09118143]
  [ 0.03590559  0.02411232 -0.01498629 ...  0.00813052 -0.00361231
    0.1030

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by Bert model<b>

In [11]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [12]:
print(features)

[array([[ 0.09233951,  0.13914399, -0.05241546, ..., -0.13947977,
        -0.0428006 ,  0.07431364],
       [ 0.11505969,  0.01998261, -0.08627468, ..., -0.00946233,
        -0.1872724 ,  0.13169006],
       [ 0.04012022,  0.05467906, -0.0424584 , ...,  0.02054804,
        -0.03819017,  0.1011811 ],
       ...,
       [ 0.08853199,  0.0228724 , -0.05121122, ..., -0.0673336 ,
        -0.03013714,  0.04913732],
       [ 0.10792391,  0.09771117, -0.05832333, ..., -0.12765658,
        -0.06493297,  0.12894136],
       [ 0.0545795 ,  0.03635893, -0.07821195, ..., -0.03016043,
        -0.06015311,  0.08903892]]), array([[ 0.0880546 ,  0.10885102,  0.06212992, ...,  0.0144938 ,
        -0.03327068, -0.0362049 ],
       [-0.03332657,  0.03442192,  0.0809955 , ...,  0.0008738 ,
        -0.02733596,  0.05783575],
       [ 0.07896441,  0.10343076, -0.01508527, ...,  0.06711898,
        -0.01733149,  0.12912118],
       [ 0.01805276,  0.07072446, -0.03629201, ..., -0.03547218,
         0.0481908 ,